MusicLM - using LucidRain's pytorch implementation. 

In [ ]:
!pip install musiclm-pytorch

In [ ]:
! pip install datasets[audio] yt-dlp

In [ ]:
#!pip install audiolm-pytorch # this mgiht be needed idk

In [54]:
!pip install encodec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [53]:
import torch
from musiclm_pytorch import MuLaN, AudioSpectrogramTransformer, TextTransformer
from encodec import EncodecModel
from encodec.utils import convert_audio
from PIL import Image
import numpy as np
import torchaudio
import torch
from IPython.display import Audio
import torchvision.transforms as transforms  
import matplotlib.pyplot as plt

# define all dataset paths, checkpoints, etc
dataset_folder = "audio_dataset"
hubert_ckpt = 'hubert/hubert_base_ls960.pt'
hubert_quantizer = f'hubert/hubert_base_ls960_L9_km500.bin' # listed in row "HuBERT Base (~95M params)", column Quantizer

In [ ]:
def get_sinewave(freq=440.0, duration_ms=200, volume=1.0, sample_rate=44100.0):
  # code adapted from https://stackoverflow.com/a/33913403
  audio = []
  num_samples = duration_ms * (sample_rate / 1000.0)
  for x in range(int(num_samples)):
    audio.append(volume * math.sin(2 * math.pi * freq * (x / sample_rate)))
  return audio

In [ ]:
# Placeholder data generation
def save_wav(file_name, audio, sample_rate=44100.0):
  # Open up a wav file
  wav_file=wave.open(file_name,"w")
  # wav params
  nchannels = 1
  sampwidth = 2
  # 44100 is the industry standard sample rate - CD quality.  If you need to
  # save on file size you can adjust it downwards. The stanard for low quality
  # is 8000 or 8kHz.
  nframes = len(audio)
  comptype = "NONE"
  compname = "not compressed"
  wav_file.setparams((nchannels, sampwidth, sample_rate, nframes, comptype, compname))
  # WAV files here are using short, 16 bit, signed integers for the 
  # sample size.  So we multiply the floating point data we have by 32767, the
  # maximum value for a short integer.  NOTE: It is theortically possible to
  # use the floating point -1.0 to 1.0 data directly in a WAV file but not
  # obvious how to do that using the wave module in python.
  for sample in audio:
      wav_file.writeframes(struct.pack('h', int( sample * 32767.0 ))) # Sort this out, this is needless
  wav_file.close()
  return

def make_placeholder_dataset():
  # Make a placeholder dataset with a few .wav files that you can "train" on, just to verify things work e2e
  if os.path.isdir(dataset_folder):
    return
  os.makedirs(dataset_folder)
  # save_wav(f"{dataset_folder}/example.wav", get_sinewave())
  # save_wav(f"{dataset_folder}/example2.wav", get_sinewave(duration_ms=500))
  os.makedirs(f"{dataset_folder}/subdirectory")
  # save_wav(f"{dataset_folder}/subdirectory/example.wav", get_sinewave(freq=330.0)) # take this line out and sine wave thing for real training attempt

make_placeholder_dataset()

# this is from working with music caps:

(https://github.com/nateraw/download-musiccaps-dataset)

In [9]:
import subprocess
import os
from pathlib import Path

from datasets import load_dataset, Audio


def download_clip(
    video_identifier,
    output_filename,
    start_time,
    end_time,
    tmp_dir='/tmp/musiccaps',
    num_attempts=5,
    url_base='https://www.youtube.com/watch?v='
):
    status = False

    command = f"""
        yt-dlp --quiet --no-warnings -x --audio-format wav -f bestaudio -o "{output_filename}" --download-sections "*{start_time}-{end_time}" {url_base}{video_identifier}
    """.strip()

    attempts = 0
    while True:
        try:
            output = subprocess.check_output(command, shell=True,
                                                stderr=subprocess.STDOUT)
        except subprocess.CalledProcessError as err:
            attempts += 1
            if attempts == num_attempts:
                return status, err.output
        else:
            break

    # Check if the video was successfully saved.
    status = os.path.exists(output_filename)
    return status, 'Downloaded'


def main(
    data_dir: str,
    sampling_rate: int = 44100,
    limit: int = None,
    num_proc: int = 1,
    writer_batch_size: int = 1000,
):
    """
    Download the clips within the MusicCaps dataset from YouTube.
    Args:
        data_dir: Directory to save the clips to.
        sampling_rate: Sampling rate of the audio clips.
        limit: Limit the number of examples to download.
        num_proc: Number of processes to use for downloading.
        writer_batch_size: Batch size for writing the dataset. This is per process.
    """

    ds = load_dataset('google/MusicCaps', split='train')
    if limit is not None:
        print(f"Limiting to {limit} examples")
        ds = ds.select(range(limit))

    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok=True, parents=True)

    def process(example):
        outfile_path = str(data_dir / f"{example['ytid']}.wav")
        status = True
        if not os.path.exists(outfile_path):
            status = False
            status, log = download_clip(
                example['ytid'],
                outfile_path,
                example['start_s'],
                example['end_s'],
            )

        example['audio'] = outfile_path
        example['download_status'] = status
        return example

    return ds.map(
        process,
        num_proc=num_proc,
        writer_batch_size=writer_batch_size,
        keep_in_memory=False
    ).cast_column('audio', Audio(sampling_rate=sampling_rate))

just dl a few, limit = 32

In [10]:
ds = main('./music_data', num_proc=2, limit=32)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/google___csv/google--MusicCaps-bedc2a0fd7888f2f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.
Limiting to 32 examples


Map (num_proc=2):   0%|          | 0/32 [00:00<?, ? examples/s]

Get samples:

In [83]:
def get_captions(idx):
    ex = ds[idx]
    return ex['caption']

In [44]:
def get_cap_wav(idx):
    ex = ds[idx]
    return ex['audio']['array'], ex['caption']

In [39]:
def get_aspects_wav(idx):
    ex = ds[idx]
    return ex['audio']['array'], ex['aspect_list']

In [81]:
wav,text = get_cap_wav(25)
print(wav,text)

[0.00675262 0.00952237 0.00806644 ... 0.01727735 0.02431311 0.        ] The low quality recording features a drum solo that consists of a double pedal kick, punchy snare, shimmering cymbals and boomy toms. It sounds energetic and manic, thanks to that kick pattern. The recording is mono and noisy.


In [50]:
wav,text = get_aspects_wav(25)
print(wav,text)

[0.00675262 0.00952237 0.00806644 ... 0.01727735 0.02431311 0.        ] ['low quality', 'mono', 'noisy', 'drums solo', 'double pedal kick', 'punchy snare', 'shimmering cymbals', 'boomy toms', 'energetic', 'manic']


In [87]:
txt1 = get_captions(12)
txt2 = get_captions(25)

In [91]:
audio_list = [txt1, txt2]

In [97]:
from musiclm_pytorch import MuLaNTrainer

#Train MuLan:

This part isn't finished, need to replace wavs and then I think it's done. 

In [47]:
wav.shape

(880832,)

In [98]:
MuLaNTrainer??

In [80]:
TextTransformer??

In [96]:
audio_transformer = AudioSpectrogramTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
    spec_n_fft = 128,
    spec_win_length = 24,
    spec_aug_stretch_factor = 0.8
)

text_transformer = TextTransformer(
    dim = 512,
    depth = 6,
    heads = 8,
    dim_head = 64,
    # raw_texts = audio_list
)

mulan = MuLaN(
    audio_transformer = audio_transformer,
    text_transformer = text_transformer
)

# get a ton of <sound, text> pairs and train

# Text transformer has raw_texts: Optional[List[str]] = None, in the forward function,
#I assume there's a way to just pass in raw texts, but not immediately obvious

# wavs = wav
# texts = text

mulan_trainer = MuLaNTrainer(
        mulan =  mulan,
        # dataset: Dataset,
        num_train_steps = 9,
        batch_size = 2,
        data_max_length = None, # ??
        folder = dataset_folder, # ds folder?
        lr = 3e-4,
        grad_accum_every = 1,
        betas = (0.9, 0.99),
        max_grad_norm = 0.5,
        valid_frac = 0.05,
        random_split_seed = 42,
        save_model_every = 1000,
        results_folder = './results',
        # accelerate_kwargs: dict = dict(), # no idea what this is, do I need to install accelerate library? 
        use_lion = False,
        force_clear_prev_results = None  # set to True | False to skip the prompt
)

wavs = torch.randn(2, 1024) # the two refers to batch size apparently
texts = audio_list #torch.randint(0, 20000, (2, 256)) # not clear on this bit does this need tokens, or the text turn to tokens first?
# so guessing the two in texts is bs as well, and max length of tokens?

# I assume this actually needs a training loop to work.

# loss = mulan(wavs, texts)
# loss.backward()

mulan_trainer.train()

# after much training, you can embed sounds and text into a joint embedding space
# for conditioning the audio LM

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 38>:38                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│ in forward:47                                                                                    │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/musiclm_pytorch/musiclm_pytorch.py:738 in forward        │
│                                                                                                  │
│   735 │   │   batch, device = wavs.shape[0], wavs.device                                         │
│   736 │   │                                                                                      │
│   737 │   │   audio_latents, audio_layers = self.get_audio_latents(wavs, return_all_layers = T   │
│ ❱ 738 │   │   text_latents, text_layers = self.get_text_latents(texts, raw_texts = raw_texts,    │
│   739 │   │                                                                                      │
│   740 │   │   if return_latents:                                                                 │
│   741 │   │   │   return audio_latents, text_latents                                             │
│ in get_text_latents:47                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/musiclm_pytorch/musiclm_pytorch.py:716 in                │
│ get_text_latents                                                                                 │
│                                                                                                  │
│   713 │   │   raw_texts: Optional[List[str]] = None,                                             │
│   714 │   │   return_all_layers = False                                                          │
│   715 │   ):                                                                                     │
│ ❱ 716 │   │   text_embeds, text_layers = self.text(texts, raw_texts = raw_texts, return_all_la   │
│   717 │   │   text_latents = self.text_to_latents(text_embeds)                                   │
│   718 │   │   out = l2norm(text_latents)                                                         │
│   719                                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _gl

In [78]:
loss = mulan(wavs, texts)
loss.backward()

In [79]:
loss

tensor(-0.0062, grad_fn=<SumBackward0>)

In [57]:
embeds = mulan.get_audio_latents(wavs)  # during training ?? what does this mean, during training vs inference?

embeds = mulan.get_text_latents(texts)  # during inference, I mean do these bits even need to run, this embeds would overwrite the one above

In [ ]:
embeds

In [ ]:
ds_path = dataset_folder

In [58]:
from musiclm_pytorch import MuLaNEmbedQuantizer
# setup the quantizer with the namespaced conditioning embeddings, unique per quantizer as well as namespace (per transformer)
quantizer = MuLaNEmbedQuantizer(
    mulan = mulan,                          # pass in trained mulan from above
    conditioning_dims = (1024, 1024, 1024), # say all three transformers have model dimensions of 1024
    namespaces = ('semantic', 'coarse', 'fine')
)
# now say you want the conditioning embeddings for semantic transformer
# wavs = torch.randn(2, 1024)
conds = quantizer(wavs = wavs, namespace = 'semantic') # (2, 8, 1024) - 8 is number of quantizers, dunno what the two is about, stereo audio?

#Semantic Transformer:

In [ ]:
from audiolm_pytorch import HubertWithKmeans, SemanticTransformer, SemanticTransformerTrainer

In [ ]:
# hubert checkpoints can be downloaded at
# https://github.com/facebookresearch/fairseq/tree/main/examples/hubert
if not os.path.isdir("hubert"):
  os.makedirs("hubert")
if not os.path.isfile(hubert_ckpt):
  hubert_ckpt_download = f"https://dl.fbaipublicfiles.com/{hubert_ckpt}"
  urllib.request.urlretrieve(hubert_ckpt_download, f"./{hubert_ckpt}")
if not os.path.isfile(hubert_quantizer):
  hubert_quantizer_download = f"https://dl.fbaipublicfiles.com/{hubert_quantizer}"
  urllib.request.urlretrieve(hubert_quantizer_download, f"./{hubert_quantizer}")

# work out if we need to be getting or dl the checkpoints if not already there
wav2vec = HubertWithKmeans(
    checkpoint_path = './hubert/hubert_base_ls960.pt',
    kmeans_path = './hubert/hubert_base_ls960_L9_km500.bin'
)

semantic_transformer = SemanticTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    dim = 1024,
    depth = 6,
    audio_text_condition = True      # this must be set to True (same for CoarseTransformer and FineTransformers)
).cuda()

trainer = SemanticTransformerTrainer(
    transformer = semantic_transformer,
    wav2vec = wav2vec,
    audio_conditioner = quantizer,   # pass in the MulanEmbedQuantizer instance above
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 9 # idk the number of training steps needed here for the semantic transformer
)

trainer.train()

# Coarse Transformer:

In [ ]:
from audiolm_pytorch import CoarseTransformer, CoarseTransformerWrapper, CoarseTransformerTrainer, FineTransformer, FineTransformerWrapper, FineTransformerTrainer, AudioLM

In [ ]:
from audiolm_pytorch import EncodecWrapper

In [ ]:
wav2vec = HubertWithKmeans(
    checkpoint_path = './hubert/hubert_base_ls960.pt',
    kmeans_path = './hubert/hubert_base_ls960_L9_km500.bin'
)

# Instantiate a pretrained EnCodec model
encodec = EncodecWrapper(
        target_sample_hz = 24000,
        strides = (2, 4, 5, 8),
        num_quantizers = 8,
)

coarse_transformer = CoarseTransformer(
    num_semantic_tokens = wav2vec.codebook_size,
    codebook_size = 1024,
    num_coarse_quantizers = 3,
    dim = 512,
    depth = 6,
    audio_text_condition = True # adding audio text cond true    
)

trainer = CoarseTransformerTrainer(
    transformer = coarse_transformer,
    codec = encodec, # change to encodec
    wav2vec = wav2vec,
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    save_results_every = 1000,
    save_model_every = 1000,
    num_train_steps = 9,
    
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason
trainer.train()

#Fine transformer

In [ ]:
encodec = EncodecWrapper(
        target_sample_hz = 24000,
        strides = (2, 4, 5, 8),
        num_quantizers = 8,
)

fine_transformer = FineTransformer(
    num_coarse_quantizers = 3, # could test 2 by 2 if using n_q = 4, or 1 by 3
    num_fine_quantizers = 5,
    codebook_size = 1024,
    dim = 512,
    depth = 6,
    audio_text_condition = True   # adding audio text cond true
)

trainer = FineTransformerTrainer(
    transformer = fine_transformer,
    codec = encodec, # change to encodec
    folder = ds_path,
    batch_size = 1,
    data_max_length = 320 * 32,
    num_train_steps = 9,
    save_model_every = 8
)
# NOTE: I changed num_train_steps to 9 (aka 8 + 1) from 10000 to make things go faster for demo purposes
# adjusting save_*_every variables for the same reason
trainer.train()

#Inference:

In [ ]:
encodec = EncodecWrapper(
        target_sample_hz = 24000,
        strides = (2, 4, 5, 8),
        num_quantizers = 8,
)

In [ ]:
# Everything together
audiolm = AudioLM(
    wav2vec = wav2vec,
    codec = encodec,
    semantic_transformer = semantic_transformer,
    coarse_transformer = coarse_transformer,
    fine_transformer = fine_transformer
)

generated_wav = audiolm(batch_size = 1)

In [ ]:
# you need the trained AudioLM (audio_lm) from above
# with the MulanEmbedQuantizer (mulan_embed_quantizer)
from musiclm_pytorch import MusicLM

musiclm = MusicLM(
    audio_lm = audio_lm, # ? do we need (batch_size = 1) here as well?
    mulan_embed_quantizer = mulan_embed_quantizer
)

music = musiclm('the crystalline sounds of the piano in a ballroom', num_samples = 4) # sample 4 and pick the top match with mulan